In [ ]:
import os
import tensorflow as tf
import numpy as np
import PIL.Image
import IPython.display as display


def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)


def load_img(path_to_img):
  max_dim = 512
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img


def imshow(image, title=None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0)

  plt.imshow(image)
  if title:
    plt.title(title)


def vgg_layers(layer_names):
  """ Creates a vgg model that returns a list of intermediate output values."""
  # Load our model. Load pretrained VGG, trained on imagenet data
  vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
  vgg.trainable = False
  
  outputs = [vgg.get_layer(name).output for name in layer_names]

  model = tf.keras.Model([vgg.input], outputs)
  return model


def gram_matrix(input_tensor):
  result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
  input_shape = tf.shape(input_tensor)
  num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
  return result/(num_locations)


class StyleContentModel(tf.keras.models.Model):
  def __init__(self, style_layers, content_layers):
    super(StyleContentModel, self).__init__()
    self.vgg =  vgg_layers(style_layers + content_layers)
    self.style_layers = style_layers
    self.content_layers = content_layers
    self.num_style_layers = len(style_layers)
    self.vgg.trainable = False

  def call(self, inputs):
    "Expects float input in [0,1]"
    inputs = inputs*255.0
    preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
    outputs = self.vgg(preprocessed_input)
    style_outputs, content_outputs = (outputs[:self.num_style_layers], 
                                      outputs[self.num_style_layers:])

    style_outputs = [gram_matrix(style_output)
                     for style_output in style_outputs]

    content_dict = {content_name:value 
                    for content_name, value 
                    in zip(self.content_layers, content_outputs)}

    style_dict = {style_name:value
                  for style_name, value
                  in zip(self.style_layers, style_outputs)}
    
    return {'content':content_dict, 'style':style_dict}


def clip_0_1(image):
  return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)


def style_content_loss(outputs, style_targets, content_targets, num_style_layers, num_content_layers, style_weight, content_weight):
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) 
                           for name in style_outputs.keys()])
    style_loss *= style_weight / num_style_layers

    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) 
                             for name in content_outputs.keys()])
    content_loss *= content_weight / num_content_layers
    loss = style_loss + content_loss
    return loss


# @tf.function()
def train_step_without_variation_loss(image, extractor, opt, style_targets, content_targets, num_style_layers, num_content_layers, style_weight, content_weight):
  with tf.GradientTape() as tape:
    outputs = extractor(image)
    loss = style_content_loss(outputs, style_targets, content_targets, num_style_layers, num_content_layers, style_weight, content_weight)

  grad = tape.gradient(loss, image)
  opt.apply_gradients([(grad, image)])
  image.assign(clip_0_1(image))


def high_pass_x_y(image):
  x_var = image[:,:,1:,:] - image[:,:,:-1,:]
  y_var = image[:,1:,:,:] - image[:,:-1,:,:]
  return x_var, y_var


def total_variation_loss(image):
  x_deltas, y_deltas = high_pass_x_y(image)
  return tf.reduce_sum(tf.abs(x_deltas)) + tf.reduce_sum(tf.abs(y_deltas))


# @tf.function()
def train_step_with_variation_loss(image, extractor, opt, style_targets, content_targets, num_style_layers, num_content_layers, style_weight, content_weight, total_variation_weight):
  with tf.GradientTape() as tape:
    outputs = extractor(image)
    loss = style_content_loss(outputs, style_targets, content_targets, num_style_layers, num_content_layers, style_weight, content_weight)
    loss += total_variation_weight*tf.image.total_variation(image)

  grad = tape.gradient(loss, image)
  opt.apply_gradients([(grad, image)])
  image.assign(clip_0_1(image))


def render_image(content_path, style_path, content_layers, style_layers, style_weight, content_weight, total_variation_weight, epochs_without_variation, epochs_with_variation, steps_per_epoch):
  num_content_layers = len(content_layers)
  num_style_layers = len(style_layers)

  content_image = load_img(content_path)
  style_image = load_img(style_path)

  extractor = StyleContentModel(style_layers, content_layers)
  style_targets = extractor(style_image)['style']
  content_targets = extractor(content_image)['content']
  opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)
  
  image = tf.Variable(content_image)
  step = 0
  for n in range(epochs_without_variation):
    for m in range(steps_per_epoch):
      step += 1
      train_step_without_variation_loss(image, extractor, opt, style_targets, content_targets, num_style_layers, num_content_layers, style_weight, content_weight)

  image = tf.Variable(content_image)
  step = 0
  for n in range(epochs_with_variation):
    for m in range(steps_per_epoch):
      step += 1
      train_step_with_variation_loss(image, extractor, opt, style_targets, content_targets, num_style_layers, num_content_layers, style_weight, content_weight, total_variation_weight)
  
  return image

In [ ]:
import IPython.display as display

content_weights = [1e1, 1e2, 1.25e2, 1.5e2, 1.75e2, 1e3, 1.5e3, 1e4, 1e5]

content_layers = ['block5_conv2', 'block4_conv2', 'block3_conv4', 'block3_conv3', 'block3_conv2'] 

style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1']

style_weight=1e-2
total_variation_weight=30
epochs_without_variation = 10
epochs_with_variation = 10
steps_per_epoch = 100

for index, content_weight in enumerate(content_weights):
  path = f"/content/weight-{index}"
  os.mkdir(path)
  for layer in content_layers:
    img = tensor_to_image(render_image("/content/content.png", "/content/style.png", [layer], style_layers, style_weight, content_weight, total_variation_weight, epochs_without_variation, epochs_with_variation, steps_per_epoch))
    img.save(f'{path}/{layer}.png')

80142336/80134624 [==============================] - 1s 0us/step
